## Equilibrium Propagation

In [13]:
# necessary imports
import os
import urllib
import struct
import numpy as np
import gzip
import cPickle

In [14]:
# download data if not available
# this part of code is directly taken from Benjamin Scellier s code
dir_path = os.path.dirname(os.path.abspath("__file__"))
path = dir_path+os.sep+"mnist.pkl.gz"

# DOWNLOAD MNIST DATASET
if not os.path.isfile(path):
    origin = ('http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz')
    print 'Downloading data from %s' % origin
    urllib.urlretrieve(origin, path)

# LOAD MNIST DATASET
f = gzip.open(path, 'rb')
(train_x_values, train_y_values), (valid_x_values, valid_y_values), (test_x_values, test_y_values) = cPickle.load(f)
f.close()
train_data_size= len(train_y_values)


In [15]:
# this part of code is directly taken from Benjamin Scellier s code
# HYPERPARAMETERS FOR A NETWORK WITH 1 HIDDEN LAYER
net1 = "net1", {
"hidden_sizes" : [500],
"n_epochs"     : 20,
"n_it_neg"     : 20,
"n_it_pos"     : 4,
"epsilon"      : np.float32(.5),
"beta"         : np.float32(.05),
"alphas"       : [np.float32(.1), np.float32(.05)]
}

In [16]:
def rho(s):
    return (1./(1.+np.exp(-s)))
def clamp(s):
    return np.clip(s, 0.,1.)


In [17]:
def to_one_hot(a, nb_class):
    one_hot=np.zeros(nb_class, dtype=np.float32)
    one_hot[a]=1.
    return one_hot

In [18]:
class ep_net(object):
    # this function is taken from Benjamn Scellier's code
    def save_params(self):
        f = file(self.path, 'wb')
        biases_values  = self.biases
        weights_values = self.weights
        to_dump        = biases_values, weights_values, self.hyperparameters, self.training_curves
        cPickle.dump(to_dump, f, protocol=cPickle.HIGHEST_PROTOCOL)
        f.close()
    def __init__(self, name, hyperparameters=dict()):
        self.path=name+".save"
        # load parameters from hyperparamerters
        self.biases, self.weights, self.hyperparameters, self.training_curves = self.__load_params(hyperparameters)
        #layer size : input layer has 28*28 neuron, hidden units has 500 neuron and output unit has 10 neurons
        layer_sizes = [28*28] + self.hyperparameters["hidden_sizes"] + [10]
        index=0
        self.x_data= train_x_values[index]
        self.train_data_size= len(train_y_values)
        self.y= train_y_values
        self.y_data= self.y[index]
        self.y_data_one_hot = to_one_hot(self.y_data, 10)
        
        values = [np.zeros((train_data_size, layer_size), dtype=np.float32) for layer_size in layer_sizes[1:]]
        #print(self.y_data_one_hot)
        
        self.layers= [self.x_data]+[value[index] for value in values]
        print(layer.shape for layer in self.layers)
    def update(self, index):
        layer_sizes = [28*28] + self.hyperparameters["hidden_sizes"] + [10]
        self.x_data= train_x_values[index]
        self.y_data= self.y[index]
        self.y_data_one_hot = to_one_hot(self.y_data, 10)
        values = [np.zeros((train_data_size, layer_size), dtype=np.float32) for layer_size in layer_sizes[1:]]
        self.layers= [self.x_data]+[value[index] for value in values]
    def total_energy(self, layers, beta):
        return self.energy(layers)+ beta* self.cost(layers)
    def cost(self, layers):
        outLayer= layers[-1]
        originalOutput= self.y_data_one_hot
        diff= outLayer- originalOutput
        return np.sum(np.multiply(diff, diff))
       
    def energy(self, layers):
        # squared norm sum of 1/2 ui^2
        # input layer x, hidden layer h, output layer y
        x= layers[0]
        h= layers[1]
        y= layers[2]
        xx= np.sum(np.multiply(clamp(x),clamp(x)))
        hh= np.sum(np.multiply(rho(h),rho(h)))
        yy= np.sum(np.multiply(rho(y),rho(y)))
        #print("value of xx")
        #print(h)
        squared_norm= xx+yy+hh
        #print(squared_norm)
        #quadratic term sum Wij rho(ui) rho (uj)
        inputWeight= self.weights[0]
        hiddenWeight= self.weights[1]
        #print(inputWeight.shape)
        #print(rho(x).reshape(1,len(x)).shape)
        #print(rho(h).shape)
        # quadratic term
        partA= rho(clamp(x)).reshape(1,len(x))
        partB= rho(h).reshape(len(h),1)
        partC= np.matmul(partA, inputWeight)
        
        partD= rho(h).reshape(1, len(h))
        partE= rho(y).reshape(len(y),1)
        partF= np.matmul(partD, hiddenWeight)
        quadratic_term= np.matmul(partC, partB)[0][0]+ np.matmul(partF, partE)[0][0]
        #quadratic_term=np.sum(np.matmul(np.matmul(rho(clamp(x)).reshape(1,len(x)),inputWeight),rho(h).reshape(len(h),1)) + np.matmul(np.matmul(rho(h).reshape(1,len(h)), hiddenWeight),rho(y).reshape(len(y),1)))
        #print(quadratic_term)
        # biased term
        inputBias= self.biases[0]
        hiddenBias= self.biases[1]
        outputBias= self.biases[2]
        #print(inputBias.shape)
        #print(rho(clamp(x)).shape)
        biased_term= np.sum(np.multiply(inputBias, rho(clamp(x))))+np.sum(np.multiply(hiddenBias, rho(h)))+np.sum(np.multiply(outputBias, rho(y)))
        
        #print(biased_term)
        #print("energy")
        #print(squared_norm)
        #print(quadratic_term.shape)
        #print(biased_term)
        #print(squared_norm+quadratic_term+biased_term)
        return (squared_norm/2.-quadratic_term/2.-biased_term)
        
    def getEnergyGradient(self, layers, weights, biases):
        x= layers[0]
        h= layers[1]
        y= layers[2]
        DE= list()
        inputWeight= weights[0]
        hiddenWeight= weights[1]
        
        inputBias= biases[0]
        hiddenBias= biases[1]
        outputBias= biases[2]
        
        
        #print("start grad")
        #print(x.shape)
        #print(h.shape)
        #print(y.shape)

        #calculate dE(x)/dx
        
        #weighted term
        partA= rho(clamp(x))
        partA_dot= np.multiply(partA, 1. - partA)
        partC= np.matmul(inputWeight, rho(h).reshape(len(h),1))
        partD= np.multiply(partA_dot.reshape(len(x),1), partC.reshape(len(x),1))
        #biased term
        
        partE= np.multiply(inputBias.reshape(len(x),1), partA_dot.reshape(len(x),1))
        dE_dx= -1*clamp(x).reshape(len(x),1)+ partD/2. + partE
        #print(dE_dx.shape)
        DE.append(dE_dx)
        #calculate  dE/dh
        partA= rho(h)
        partB= rho(clamp(x))
        partC= np.matmul(partB.reshape(1, len(x)), inputWeight)
        partD= np.multiply(partA,1.-partA)
        partE= np.multiply(partC.reshape(len(h),1), partD.reshape(len(partD),1)) #first w term
        
        partF= np.matmul(hiddenWeight, rho(y).reshape(len(y),1))
        partG= np.multiply(partF, partD.reshape(len(h),1)) # second w term
        #biased term
        partH= np.multiply(hiddenBias.reshape(len(h),1),partD.reshape(len(h),1) )
        
        dE_dh= -1*h.reshape(len(h),1) + (partE+partG)/2. + partH
        #print(dE_dh.shape)
        
        DE.append(dE_dh)
        
        #calculate dE/dy
        partA= rho(y)
        partB= np.multiply(partA, 1. - partA)
        partC= np.matmul(rho(h).reshape(1, len(h)),hiddenWeight)
        partD =np.multiply(partC.reshape(len(y),1), partB.reshape(len(y),1))# weighted part
        
        partE= np.multiply(outputBias.reshape(len(y),1), partB.reshape(len(y),1))
        
        dE_dy= -1*y.reshape(len(y), 1)+ partD/2. + partE
        DE.append(dE_dy)
        #print("end grad")
        return DE
        
    def free_phase(self, n_iteration, epsilon, index):
        #print("in ffre phse")
        layers= self.layers
        for i in range(n_iteration):
            layers_grad= self.getEnergyGradient(layers, self.weights, self.biases)
            layers_new= [layers[0]]+ [layer+epsilon*grad.reshape(len(grad)) for layer, grad in zip(layers, layers_grad)[1:]] 
            layers= layers_new
            

        layers_end= [layer[-1] for layer in layers]
        
        E= self.energy(layers)
        C= self.cost(layers)
        y_prediction= np.argmax(layers[-1])
        error = np.not_equal(y_prediction, self.y_data)*1.

        # return the measure E, c and error
        return E,C,error, layers
    def weakly_clamped_phase(self, n_it_pos, epsilon, beta, alphas, layers):
        #print(n_it_pos)
        #print(epsilon)
        #print(beta)
        #print(alphas)
        # layer of free phase
        
        free_phase_layers= layers
        for i in range(n_it_pos):
            F=self.total_energy(layers, beta)
            F_grad= self.getTotalGradient(self.layers, self.weights, self.biases, epsilon, beta, alphas)
            layers_new= [layers[0]]+ [layer-epsilon*grad.reshape(len(grad)) for layer, grad in zip(layers, F_grad)[1:]]
            layers= layers_new
        weakly_clamped_layers= layers
        
        biases_new= list()
        # update the weights and biases
        biases_grad= self.getBiasesGrad(free_phase_layers, weakly_clamped_layers)
        weights_grad= self.getWeightsGrad(free_phase_layers, weakly_clamped_layers)
        biases_new2  = [b - alpha * dot/beta for b,alpha,dot in zip(self.biases[1:],alphas,biases_grad[1:])]
        biases_new.append(self.biases[0]-alphas[0]*biases_grad[0]/beta)
        biases_new.append(biases_new2[0])
        biases_new.append(biases_new2[1])
        weights_new = [W - alpha * dot/beta for W,alpha,dot in zip(self.weights,   alphas,weights_grad)]
        
        #another view
        delW= self.getDelW(weakly_clamped_layers, free_phase_layers, beta)
        weights_new= [W - alpha * dot for W,alpha,dot in zip(self.weights,   alphas,delW)]
        return biases_new, weights_new
    
    def getDelW(self, wLayer, fLayer, beta):
        wIn= wLayer[0]
        wHid= wLayer[1]
        wOut= wLayer[2]
        fIn= fLayer[0]
        fHid= fLayer[1]
        fOut= fLayer[2]
        partA= np.matmul(wIn.reshape(len(wIn),1),wHid.reshape(1, len(wHid)))/beta
        partB = np.matmul(wHid.reshape(len(wHid),1),wOut.reshape(1, len(wOut)))/beta
        
        partC= np.matmul(fIn.reshape(len(fIn),1),fHid.reshape(1, len(fHid)))/beta
        partD = np.matmul(fHid.reshape(len(fHid),1),fOut.reshape(1, len(fOut)))/beta
        
        delW= list()
        delW.append(partA-partC)
        delW.append(partB-partD)
        
        
        return delW
    def getWeightsGrad(self, fLayers,wLayers):
        dF_dw=list()
        #dF_dw0
        partA= rho(clamp(fLayers[0])).reshape(len(fLayers[0]),1)*rho(fLayers[1]).reshape(1, len(fLayers[1]))
        partB= rho(clamp(wLayers[0])).reshape(len(wLayers[0]),1)*rho(wLayers[1]).reshape(1, len(wLayers[1]))
        
        partC= rho(fLayers[1]).reshape(len(fLayers[1]),1)*rho(fLayers[2]).reshape(1, len(fLayers[2]))
        partD= rho(wLayers[1]).reshape(len(wLayers[1]),1)*rho(wLayers[2]).reshape(1, len(wLayers[2]))
        dF_dw0= (partA-partB)/2.
        #print("weigths grad")
        dF_dw1= (partC-partD)/2.
        #print(dF_dw0.shape)
        dF_dw.append(dF_dw0)
        dF_dw.append(dF_dw1)
        return dF_dw
    def getBiasesGrad(self,fLayers, wLayers):
        dF_db= list()
        inputBias= self.biases[0]
        hiddenBias=self.biases[1]
        outputBias=self.biases[2]
        #dF/db0
        dF_db0= rho(clamp(fLayers[0]))-rho(clamp(wLayers[0]))
        dF_db1= rho(fLayers[1])-rho(wLayers[1])
        dF_db2= rho(fLayers[2])-rho(wLayers[2])
        dF_db.append(dF_db0)
        dF_db.append(dF_db1)
        dF_db.append(dF_db2)
        
        return dF_db
        
    def getTotalGradient(self, layers, weights, biases, epsilon, beta, alphas):
        DF= self.getEnergyGradient(layers, weights, biases)
        #modify last gradient
        ll= DF[-1]
        #print("total grad")
        #print(ll.shape)
        y_prediction= layers[-1]
        DF[-1]=DF[-1]-2*beta*(self.y_data_one_hot.reshape(len(self.y_data_one_hot),1)-y_prediction.reshape(len(y_prediction),1))
        #print(DF[-1].shape)
        return DF
    def __load_params(self, hyperparameters):
        hyper = hyperparameters
        # Glorot/Bengio weight initialization
        def initialize_layer(n_in, n_out):
            rng = np.random.RandomState()
            W_values = np.asarray(
                rng.uniform(
                    low=-np.sqrt(6. / (n_in + n_out)),
                    high=np.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=np.float32
            )
            return W_values

        if os.path.isfile(self.path):
            f = file(self.path, 'rb')
            biases_values, weights_values, hyperparameters, training_curves = cPickle.load(f)
            f.close()
            for k,v in hyper.iteritems():
                hyperparameters[k]=v
        else:
            layer_sizes = [28*28] + hyperparameters["hidden_sizes"] + [10]
            biases_values  = [np.zeros((size,), dtype=np.float32) for size in layer_sizes]
            weights_values = [initialize_layer(size_pre,size_post) for size_pre,size_post in zip(layer_sizes[:-1],layer_sizes[1:])]
            training_curves = dict()
            training_curves["training error"]   = list()
            training_curves["validation error"] = list()

        biases  = [value for value in biases_values]
        weights = [value for value in weights_values]

        return biases, weights, hyperparameters, training_curves

        

In [21]:
#train the model
def train_model(net):
    path         = net.path
    hidden_sizes = net.hyperparameters["hidden_sizes"]
    n_epochs     = net.hyperparameters["n_epochs"]
    n_it_neg     = net.hyperparameters["n_it_neg"]
    n_it_pos     = net.hyperparameters["n_it_pos"]
    epsilon      = net.hyperparameters["epsilon"]
    beta         = net.hyperparameters["beta"]
    alphas       = net.hyperparameters["alphas"]
    # for each pair of x,y 
    Err=np.zeros(train_data_size, dtype=np.float32)
    for a in range(n_epochs):
        print(train_data_size)
        for i in range(train_data_size):
            #update self.layers
            #call free phase
            if i%1000==0:
                print("1000th Milestone")
            E,C,error, layers=net.free_phase(n_it_neg, epsilon, i)
            Err[i]= error*100.
            # run free phase till settles to a fixed point and collect rho(u0)s 
            #E,C,error = net.free_phase(n_it_neg, epsilon)
            # run weakly clamped phase  and collect rho(ubeta)
            
            sign = 2*np.random.randint(0,2)-1 # random sign +1 or -1
            beta = np.float32(sign*beta) # choose the sign of beta at random
            biases_new, weights_new= net.weakly_clamped_phase(n_it_pos, epsilon, beta, alphas, layers)
            
            # update W
            net.biases[0]= biases_new[0]
            net.biases[1]=biases_new[1]
            net.biases[2]=biases_new[2]
            net.weights[0]= weights_new[0]
            net.weights[1]= weights_new[1]
            
            #update layers 
            if (i<train_data_size-1):
                net.update(i+1)
        print("processed "+str(i)+"images")
        print("epoch "+ str(a)+" completed")
        net.training_curves["training error"].append(np.average(Err))
        print(net.training_curves["training error"])
    net.save_params()
    print("training ends")

In [24]:
import time
start_time = time.clock()
train_model(ep_net(*net1))
duration = (time.clock() - start_time) / 60.
print("   duration=%.1f min" % (duration))

<generator object <genexpr> at 0x7f3c2c0fdc30>
50000
1000th Milestone


/home/backbencher/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app
/home/backbencher/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: RuntimeWarning: overflow encountered in multiply
/home/backbencher/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:208: RuntimeWarning: overflow encountered in divide
/home/backbencher/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:212: RuntimeWarning: invalid value encountered in subtract


1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
processed 49999images
epoch 0 completed
[90.138]
50000
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Milestone
1000th Mil